In [1]:
!pwd

/cluster/work/cotterell/kdu/measureLM


In [2]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [12]:
DATA_ROOT = "data/YagoECQ/"
YAGO_GPT_FAKE_ENTITIES_PATH = os.path.join(DATA_ROOT, "chatgpt_fake_entities_all.csv") 
YAGO_QEC_PATH = os.path.join(DATA_ROOT, "yago_qec.json")

In [4]:
openai.api_key = os.getenv("OPENAI_API_KEY")


In [5]:
client = openai.OpenAI(api_key=openai.api_key)
print(client.models.list())
# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=["hello"],
#     temperature=0,
#     max_tokens=1024
# )

SyncPage[Model](data=[Model(id='gpt-3.5-turbo-1106', created=1698959748, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct', created=1692901427, object='model', owned_by='system'), Model(id='davinci-002', created=1692634301, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-4-0613', created=1686588896, ob

In [6]:
entity_class_to_prompt = {
    "Organization": "Give me a list of {} names of organizations that are not real, but could sound plausibly real.",
    "Place": "Give me a list of {} names of places that are not real, but could sound plausibly real.",
    "Event": "Give me a list of {} names of events that are not real, but could sound plausibly real.",
    # "Event": "Give me a list of {} names of historical events that are not real, but could sound plausibly real.",
    "Person": "Give me a list of {} names of people who could exist but don't. Try to make them different from each other, e.g. avoid repeating surnames or names from the same category. Give me some from different cultures and time periods. Avoid any names of real people, and especially of famous people. Avoid any names that sound overly fantastical.",
    # "Person": "Give me a list of 1000 names of people that are not real, but could sound plausibly real.",
    "Product": "Give me a list of {} names of products that are not real, but could sound plausibly real.",
    "CreativeWork": "Give me a list of {} total names of 20% books/poems, 20% songs, 20% art pieces, 20% theater shows, and 20% music pieces that are not real, but could sound plausibly real. Try to make them different from each other. Give me some from different cultures, time periods, genres, and mediums. Avoid any names of real works, and especially of famous works. Avoid overly fantastical sounding names.",
    # "CreativeWork": "Give me a list of 1000 names of books, musicals, songs, poems, theater shows, and music pieces that are not real, but could sound plausibly real. Try to make them different from each other. Give me some from different cultures, time periods, genres, and mediums. Avoid any names of real works, and especially of famous works. Avoid overly fantastical sounding names.",
    # "CreativeWork": "Give me a list of 1000 names of books, musicals, songs, poems, theater shows, and music pieces that are not real, but could sound plausibly real. Try to make them different from each other. Give me some from different cultures, time periods, genres, and mediums. Avoid any names of real works, and especially of famous works.",
    # "CreativeWork": "Give me a list of 1000 names of creative works (including literature, art, music, film, and theater) that are not real, but could sound plausibly real.",
}

In [7]:
from collections import defaultdict
def generate_fake_entity_names(prompt, model="gpt-4-1106-preview"):
    print(f"Calling {model} on the following prompt:\n{prompt}")
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a creative assistant. Your purpose is to help the user come up with fictional names fitting the user's criteria. Give the user the requested number of names, no matter how long the response gets or how many names the user asks for. If the user asks for 1000 names, give 1000 names and do not split the list into multiple responses and give the full list of 1000 names. Do not repeat names."},
            {"role": "user", "content": prompt}
        ]
    )
    
    import re
    # Use the regex to find all matches
    ents = re.findall(r'^\d+\.\s(.+)$', response.choices[0].message.content, re.MULTILINE)
    return ents

def generate_fake_entity_names_df(entity_class_to_prompt, num_rows=1000):
    entity_class_to_entities = defaultdict(set)
    for entity_class, prompt in entity_class_to_prompt.items():
        print("Generating {} entities...".format(entity_class))
        while len(entity_class_to_entities[entity_class]) < num_rows:
            ents = generate_fake_entity_names(prompt.format(num_rows))
            entity_class_to_entities[entity_class] = entity_class_to_entities[entity_class].union(set(ents))
            print("\tGenerated {} / {} {}s".format(len(entity_class_to_entities[entity_class]), num_rows, entity_class))

        entity_class_to_entities[entity_class] = list(entity_class_to_entities[entity_class])[:num_rows]
    
    return pd.DataFrame(entity_class_to_entities)

# def clean_ragged_df(df, max_len=1000):
#     new_df = pd.DataFrame()
#     for col in df.columns:
#         new_df[col] = df[col].dropna().drop_duplicates().sample(max_len).tolist()

#     return new_df

In [8]:
gpt_fake_entities_df = generate_fake_entity_names_df(entity_class_to_prompt, num_rows=1000)
gpt_fake_entities_df

Generating Organization entities...
Calling gpt-4-1106-preview on the following prompt:
Give me a list of 1000 names of organizations that are not real, but could sound plausibly real.
	Generated 559 / 1000 Organizations
Calling gpt-4-1106-preview on the following prompt:
Give me a list of 1000 names of organizations that are not real, but could sound plausibly real.
	Generated 1108 / 1000 Organizations
Generating Place entities...
Calling gpt-4-1106-preview on the following prompt:
Give me a list of 1000 names of places that are not real, but could sound plausibly real.
	Generated 707 / 1000 Places
Calling gpt-4-1106-preview on the following prompt:
Give me a list of 1000 names of places that are not real, but could sound plausibly real.
	Generated 1294 / 1000 Places
Generating Event entities...
Calling gpt-4-1106-preview on the following prompt:
Give me a list of 1000 names of events that are not real, but could sound plausibly real.
	Generated 504 / 1000 Events
Calling gpt-4-1106-pr

,Organization,Place,Event,Person,Product,CreativeWork
0,Radiant Heat HVAC,Jewelcrest,Fusion Taste Fest,DeShawn Merritt,HeatHugger Portable Space Heater,Embered Horizons
1,Vortex Visual Arts Studio,Midsummer Meadow,Teaching With Technology Symposium,Zander Fuchs,FreshMint Toothpaste Dispenser,Harvest’s Bounty
2,Lapis Lazuli Fine Arts Gallery,Quilted Quay,Levitation Magic Workshop,Xiao Wei,BreezeBlast Portable AC,The Shattered Visage
3,Zenith Zeniths,Yondershire,Nautical Neptune Festival,Stefanos Georgiou,WillowWisp Aromatherapy Diffusers,The Laurel’s Secret
4,Opaline Beauty Corp,Highrock,Brew & Brine Pickle and Beer Fest,Risto Väisänen,PureStream Water Softener,Beneath the Wisteria
...,...,...,...,...,...,...
995,Flourish Frontier,Rivermoor,Unseen Universe Virtual Reality Expo,Avi Goldberg,FireFly Lanterns,The Crown of Reverie
996,Gatekeeper Guild,Amberglow,The Experiential Traveler Conference,Zenon Kowalski,HandyHang Picture Hanging Tool,The Aether Fantasia
997,Polar Climate Initiative,Icicle Isle,Luxury Travel and Experiences Expo,Batool Hussain,CoolCalm Personal Fans,The Vermillion Labyrinth
998,Expanse Wide-Area Networking,Drakestone,Ethereal Electronic Arts Forum,Eduard Strauss,BubbleBlast Automatic Bubble Machine,Anthem for the Amber Amulet


In [9]:
gpt_fake_entities_df.to_csv("data/YagoECQ/gpt_fake_entities_all.csv", index=False)

In [6]:
# response = client.chat.completions.create(
#   model="gpt-4-1106-preview",
#   messages=[
#     {"role": "system", "content": "You are a creative assistant. Your purpose is to help the user come up with fictional names fitting the user's criteria. Give the user the requested number of names, no matter how long the response gets or how many names the user asks for. If the user asks for 1000 names, give 1000 names and do not split the list into multiple responses and give the full list of 1000 names. Do not repeat names."},
#     {"role": "user", "content": entity_class_to_prompt["Event"]}
#   ]
# )

In [7]:
# import re
# # Use the regex to find all matches
# ents = re.findall(r'^\d+\.\s(.+)$', response.choices[0].message.content, re.MULTILINE)
# len(ents), len(set(ents)), ents

(391,
 391,
 ['The Battle of Shadowed Vale',
  'The Treaty of Vermillion Meadows',
  'The Great River Uprising',
  'The Siege of Stonehelm',
  'The March of the Iron Legion',
  'The Fall of Crimson Keep',
  'The Emerald Plague',
  "The Peasants' Dawn Revolt",
  'The Second Moonlight Concord',
  'The Burning of Whitebridge',
  'The Sapphire Edict',
  "The Skirmish at Wolf's Crest",
  "The Highland Clans' Accord",
  'The Sundering of the Seven Stars',
  'The Night of Whispering Shadows',
  'The Parchment War',
  'The Silent Siege of Dunraven',
  'The Velvet Revolution',
  'The Frost Rebellion',
  'The Coronation Crisis',
  'The Jade Treaty',
  'The Defiance of Blackwater',
  'The Amber Accord',
  "The Oracle's Prophecy Fulfillment",
  'The Great Schism of Elarian',
  'The Siege of the Sunken Citadel',
  'The Twilight Insurrection',
  'The Red Harvest',
  'The Breaking of the Bronze Gate',
  "The Dragon's Maw Standoff",
  'The Summit at Mount Gallantry',
  'The Onyx Oath',
  'The Shatteri

In [49]:
# PATH = "data/YagoECQ/gpt_fake_entities.csv"
# try:
#     df = pd.read_csv(PATH)
#     df2 = pd.DataFrame(ents, columns=["CreativeWork"])
#     df = pd.concat([df, df2])
# except:
#     print("uhoh")
#     df = pd.DataFrame(ents, columns=["CreativeWork"])

# len(df["Event"].drop_duplicates()), len(df["Event"].dropna())

(502, 501)

In [41]:
# new_df = pd.DataFrame()
# for col in df.columns:
#     new_df[col] = df[col].dropna().drop_duplicates().sample(100).tolist()

# new_df

,Organization,Place,Event,Product,Person,CreativeWork
0,Pulsar Particle Physics Lab,Echobank Lake,Epiphany Engineering Exhibition,SavorStove Pressure Cooker,Winona LaDuke,Turbulent Tides and Tenors
1,Memento Memorabilia Co.,Mossbank Grove,Aquatic Animal Animation Workshop,EasyPulse Oximeter,Yannis Pappas,The Cosmogonic Requiem
2,Elemental Resource Management,Dreamcatcher Meadows,Clandestine Circus,BreezePro Ceiling Fan,Ryoji Chiba,The Noctilucent Night
3,Veritas Research and Development,Pinefall Gulch,Pulsar Pop Culture Convention,PediPerfect Foot Massager,Bela Kovács,Chime of the Charmed Chalice
4,Snapdragon Floral Design,Cindercloud Forest,Cybersecurity Conclave,UltraPrint 3D Printer,Jaskaran Singh,The Reckoning of the Rosewood Rings
...,...,...,...,...,...,...
95,Aurora Borealis Research Institute,Cliffside Sanctuary,Zenith Zen Garden Exhibition,MightyClean Handheld Vacuum,Kalpana Ghimire,The Compact of the Celestial Chains
96,Ambrosia Health and Beauty Spa,Lanternfall Woods,Sprite Sprint 5K,ZenSleep Pillow Spray,Querida Salazar,The Shadows Over Alsvartr
97,Keystone Diagnostics,Redspire Grove,Palette Pleasure Art Walk,SqueezeEase Tube Squeezer,Ulyana Kozlova,The Veiled Ascendancy
98,Resonance Acoustic Engineering,Merrowhaven,Tropic Twist Festival,DreamBlend Smoothie Maker,Varun Sehgal,The Obscured Revenant


In [27]:
# df.to_csv("data/YagoECQ/gpt_fake_entities.csv", index=False)

In [30]:
###########################################
# 6. Construct fake entities with chatgpt #
###########################################
import json
import pandas as pd
import random
import numpy as np
# import torch
import itertools

def set_seed(seed=0):
    # torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

set_seed(0)
fake_entities_gpt_df = pd.read_csv(YAGO_GPT_FAKE_ENTITIES_PATH)
fake_entities_gpt_df = fake_entities_gpt_df.add_prefix("http://schema.org/")

# Load yago_qec
with open(YAGO_QEC_PATH, "r") as fp:
    yago_qec = json.load(fp)

# Randomly sample fake entities that are eligible according to entity type for each relation and save to yago_qec
for k, v in yago_qec.items():
    entity_types = yago_qec[k]["entity_types"]
    eligible_fake_entities = list(itertools.chain.from_iterable([fake_entities_gpt_df[et] for et in entity_types]))
    yago_qec[k]["gpt_fake_entities"] = random.sample(eligible_fake_entities, min(len(yago_qec[k]["entities"]), len(eligible_fake_entities)))

# Save yago_qec including fake entities
with open(YAGO_QEC_PATH, "w", encoding='utf-8') as fp:
    json.dump(yago_qec, fp, ensure_ascii=False, indent=4)

In [33]:
k, yago_qec[k]["entity_types"], yago_qec[k]["gpt_fake_entities"]

('reverse-http://schema.org/homeLocation',
 ['http://schema.org/Organization', 'http://schema.org/Place'],
 ['Northwind Apparel',
  'Infinity Investment Strategies',
  'Genesis Guild',
  'Ivyfall',
  'Groovy Smoothie Bar',
  'Ethereal Shores',
  'Cavernhold',
  'Prodigious Pathfinding',
  'Aether Array',
  'Omniscient Ops',
  'Viridale',
  'Glacial Crest',
  'Newdawn City',
  'Havencrest',
  'Knollwood',
  'Sunblaze',
  'Ion Electric Car Rentals',
  'Tempest Bay',
  'Ignite Fire Safety Consultants',
  'Zephyr Heights',
  'Ethos Enterprises',
  'Triton Trench',
  'Velocity Athletics Equipment',
  'Compass Consortia',
  'Meridian Outreach',
  'Zodiac Aerospace Dynamics',
  'Luminar Legion',
  'Alpine Alliance',
  'Antediluvian Arboretum',
  'Pink Peony Cosmetics',
  'Synergy Synergy',
  'Juniper Organic Foods',
  'Rosetta Realm',
  'Silver Sphere Systems',
  'Cobaltcanyon',
  'Verity Ventures',
  'Nightingale Nests',
  'Tempo Tech',
  'Citrine Bay',
  'Waverwind Watch',
  'Lone Oak',
  '

In [68]:
from typing import Dict

ENTITY_URI_TO_DEGREE_AMBIGUOUS_PATH = os.path.join(DATA_ROOT, "entity_uri_to_degree_including_ambiguous_entities.json")
with open(ENTITY_URI_TO_DEGREE_AMBIGUOUS_PATH, "r") as fp:
    entity_uri_to_degree_including_ambiguous_entities = json.load(fp)

ENTITY_NAME_TO_POSSIBLE_ENTITY_URIS_PATH = os.path.join(DATA_ROOT, "entity_name_to_possible_entity_uris.json")
with open(ENTITY_NAME_TO_POSSIBLE_ENTITY_URIS_PATH, "r") as fp:
    entity_name_to_possible_entity_uris = json.load(fp)
    
ENTITY_NAMESAKE_TO_DEGREE_PATH = os.path.join(DATA_ROOT, "entity_namesake_to_degree.json")
entity_namesake_to_degree: Dict[str, int] = {
    k: sum([int(entity_uri_to_degree_including_ambiguous_entities[uri]) for uri in uris]) for k, uris in entity_name_to_possible_entity_uris.items()
}
with open(ENTITY_NAMESAKE_TO_DEGREE_PATH, "w", encoding='utf-8') as fp:
    json.dump(entity_namesake_to_degree, fp, ensure_ascii=False, indent=4)

ENTITY_NAMESAKE_TO_NUM_URIS_PATH = os.path.join(DATA_ROOT, "entity_namesake_to_num_uris.json")
entity_namesake_to_num_uris: Dict[str, int] = {
    k: len(uris) for k, uris in entity_name_to_possible_entity_uris.items()
}
with open(ENTITY_NAMESAKE_TO_NUM_URIS_PATH, "w", encoding='utf-8') as fp:
    json.dump(entity_namesake_to_num_uris, fp, ensure_ascii=False, indent=4)

In [65]:
len(entity_namesake_to_degree), len(entity_name_to_possible_entity_uris), list(entity_namesake_to_degree.items())[:10], list(entity_namesake_to_num_uris.items())[:10]

(45040,
 45040,
 [('Paul McCartney', 894),
  ('Paul Allen', 146),
  ('Joan Warburton', 48),
  ('Artvin Province', 313),
  ('Grigory Zass', 45),
  ('Jimmy McHugh', 157),
  ('Panteleimon Romanov', 72),
  ('Vliet Street Commons', 11),
  ('Yolande Donlan', 93),
  ('Framatome', 62)],
 [('Paul McCartney', 1),
  ('Paul Allen', 3),
  ('Joan Warburton', 1),
  ('Artvin Province', 1),
  ('Grigory Zass', 1),
  ('Jimmy McHugh', 1),
  ('Panteleimon Romanov', 1),
  ('Vliet Street Commons', 1),
  ('Yolande Donlan', 1),
  ('Framatome', 1)])

In [72]:
# Load yago entity namesake to degree stats (including all entity uris sharing that namesake)
ENTITY_NAMESAKE_TO_DEGREE_PATH = os.path.join(DATA_ROOT, "entity_namesake_to_degree.json")
with open(ENTITY_NAMESAKE_TO_DEGREE_PATH, "r") as fp:
    entity_namesake_to_degree = json.load(fp)
    
for k, v in yago_qec.items():
    yago_qec[k]["entity_namesake_to_degree"] = [
        int(entity_namesake_to_degree[entity]) for entity in yago_qec[k]["entities"]
    ]
    
# Save yago_qec including fake entities
with open(YAGO_QEC_PATH, "w", encoding='utf-8') as fp:
    json.dump(yago_qec, fp, ensure_ascii=False, indent=4)

k, sorted(list(zip(yago_qec[k]["entity_namesake_to_degree"], yago_qec[k]["entities"])), key=lambda x: x[0])[:10]

('reverse-http://schema.org/homeLocation',
 [(9, 'Wingerworth Hall'),
  (13, 'Farragut Houses'),
  (14, 'Edith Marion Patch House'),
  (14, "Michigan Governor's Mansion"),
  (15, 'Allestree Hall'),
  (16, 'Doughty House'),
  (16, 'Garden City Hotel'),
  (17, 'Pin Mill'),
  (18, 'Le Formentor'),
  (18, 'Jiangzhou')])

In [54]:
# Load yago entity uri to degree stats
ENTITY_URI_TO_DEGREE_PATH = os.path.join(DATA_ROOT, "entity_uri_to_degree.json")
with open(ENTITY_URI_TO_DEGREE_PATH, "r") as fp:
    entity_uri_to_degree = json.load(fp)
    
for k, v in yago_qec.items():
    yago_qec[k]["entity_uri_to_degree"] = [
        int(entity_uri_to_degree[uri]) for uri in yago_qec[k]["entity_uris"]
    ]
    
# Save yago_qec including fake entities
with open(YAGO_QEC_PATH, "w", encoding='utf-8') as fp:
    json.dump(yago_qec, fp, ensure_ascii=False, indent=4)

k, sorted(list(zip(yago_qec[k]["entity_uri_to_degree"], yago_qec[k]["entities"])), key=lambda x: x[0])[-10:]

In [75]:
# Load yago entity namesake to number of different uris
ENTITY_NAMESAKE_TO_NUM_URIS_PATH = os.path.join(DATA_ROOT, "entity_namesake_to_num_uris.json")
with open(ENTITY_NAMESAKE_TO_NUM_URIS_PATH, "r") as fp:
    entity_namesake_to_num_uris = json.load(fp)
    
for k, v in yago_qec.items():
    yago_qec[k]["entity_namesake_to_num_uris"] = [
        int(entity_namesake_to_num_uris[entity]) for entity in yago_qec[k]["entities"]
    ]
    
# Save yago_qec including fake entities
with open(YAGO_QEC_PATH, "w", encoding='utf-8') as fp:
    json.dump(yago_qec, fp, ensure_ascii=False, indent=4)

k, sorted(list(zip(yago_qec[k]["entity_namesake_to_num_uris"], yago_qec[k]["entities"])), key=lambda x: x[0])[-10:]

('reverse-http://schema.org/homeLocation',
 [(58, 'Paris'),
  (58, 'Paris'),
  (58, 'Paris'),
  (58, 'Paris'),
  (63, 'Springfield'),
  (64, 'Waterloo'),
  (69, 'Franklin'),
  (81, 'Victoria'),
  (174, 'Union Township'),
  (199, 'Washington Township')])